In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 50% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_50, _, y_50 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.5, stratify=y_label_0_1, random_state=42)

In [6]:
X_50.shape

(955012, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt50 = DecisionTreeClassifier()
# dt50.fit(X_50, y_50)

xgb
RF


RandomForestClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 2 3 4 5] [    12     18      1      1 238721]
(12, 42) (12,)
Save 1 to baseline_Def2.npy
(18, 42) (18,)
Save 2 to baseline_Def3.npy
(1, 42) (1,)
Save 3 to baseline_Def4.npy
(1, 42) (1,)
Save 4 to baseline_Def5.npy
(238721, 42) (238721,)
Save 5 to baseline_Def6.npy
Execution Time: 1.362762 seconds
(716259,)
[0 2 4 5 8 9] [411895 292179    386   5003   6704     92]
(411895, 42) (411895,)
Save 0 to BIM_Def1.npy
(292179, 42) (292179,)
Save 2 to BIM_Def3.npy
(386, 42) (386,)
Save 4 to BIM_Def5.npy
(5003, 42) (5003,)
Save 5 to BIM_Def6.npy
(6704, 42) (6704,)
Save 8 to BIM_Def9.npy
(92, 42) (92,)
Save 9 to BIM_Def10.npy
Execution Time: 2.996732 seconds
(716259,)
[0 1 2 4 5 7 8 9] [453926      4 241404   1135  10506   1027   4299   3958]
(453926, 42) (453926,)
Save 0 to FGSM_Def1.npy
(4, 42) (4,)
Save 1 to FGSM_Def2.npy
(241404, 42) (241404,)
Save 2 to FGSM_Def3.npy
(1135, 42) (1135,)
Save 4 to FGSM_Def5.npy
(10506, 42) (10506,)
Save 5 to FGSM_Def6.npy
(1027, 42) (1027,)
Save 7 to

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 2 3 4 5] [     5     14      1      1 238732]
(5, 42) (5,)
Save 1 to baseline_Def2.npy
(14, 42) (14,)
Save 2 to baseline_Def3.npy
(1, 42) (1,)
Save 3 to baseline_Def4.npy
(1, 42) (1,)
Save 4 to baseline_Def5.npy
(238732, 42) (238732,)
Save 5 to baseline_Def6.npy
Execution Time: 4.360758 seconds
(716259,)
[0 2 4 5 9] [404526 308744     52   2936      1]
(404526, 42) (404526,)
Save 0 to BIM_Def1.npy
(308744, 42) (308744,)
Save 2 to BIM_Def3.npy
(52, 42) (52,)
Save 4 to BIM_Def5.npy
(2936, 42) (2936,)
Save 5 to BIM_Def6.npy
(1, 42) (1,)
Save 9 to BIM_Def10.npy
Execution Time: 13.869575 seconds
(716259,)
[0 2 4 5 9] [219228 256477   1730 235830   2994]
(219228, 42) (219228,)
Save 0 to FGSM_Def1.npy
(256477, 42) (256477,)
Save 2 to FGSM_Def3.npy
(1730, 42) (1730,)
Save 4 to FGSM_Def5.npy
(235830, 42) (235830,)
Save 5 to FGSM_Def6.npy
(2994, 42) (2994,)
Save 9 to FGSM_Def10.npy
Execution Time: 11.805654 seconds
(716259,)
[0 2 4 5 9] [404526 308744     52   2936      1]
(404526, 

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt50.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ExcludeCaFA/WUSTL_Input50/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ExcludeCaFA/WUSTL_Input50/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ExcludeCaFA/WUSTL_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_ExcludeCaFA.csv")

In [13]:
all_attack_list

0       1      2       3   4      5       6   7     8      9     10  \
0   XGB50       0     12      18   1      1  238721   0     0      0     0   
1   XGB50  411895      0  292179   0    386    5003   0     0   6704    92   
2   XGB50  453926      4  241404   0   1135   10506   0  1027   4299  3958   
3   XGB50  411895      0  292179   0    386    5003   0     0   6704    92   
4   XGB50      88      0    6330   0  10284  687506   0  3347    167  8537   
5   XGB50  216334     27  384017   0    705   47204   0   200  67732    40   
6   XGB50       0     37      50   3      4  716165   0     0      0     0   
7   XGB50   74065  29213  288021   6    842  319387   0     0   3996   729   
8   XGB50   25328      0  685468   0      9    1692   0     0   3762     0   
9   XGB50  450663      0  248631   2    113    4119   0     0  12718    13   
10   RF50       0      5      14   1      1  238732   0     0      0     0   
11   RF50  404526      0  308744   0     52    2936   0     0      0     1   
12   RF50  219228      0  256477   0   1730  235830   0     0      0  2994   
13   RF50  404526      0  308744   0     52    2936   0     0      0     1   
14   RF50      80      0    6260  34   8560  687773  14  3380    180  9978   
15   RF50  162199      0  425424   0     19  128616   0     0      0     1   
16   RF50       0     15      38   3      3  716200   0     0      0     0   
17   RF50      55      0  528090   0      0  188114   0     0      0     0   
18   RF50    1769      0  702328   0      0   12162   0     0      0     0   
19   RF50  380700      0  264647  12    175   70635   0     0      0    90   
20   DT50       0      0       0   0      0       0   0     0      0     0   
21   DT50       0      0       0   0      0       0   0     0      0     0   
22   DT50       0      0       0   0      0       0   0     0      0     0   
23   DT50       0      0       0   0      0       0   0     0      0     0   
24   DT50       0      0       0   0      0       0   0     0      0     0   
25   DT50       0      0       0   0      0       0   0     0      0     0   
26   DT50       0      0       0   0      0       0   0     0      0     0   
27   DT50       0      0       0   0      0       0   0     0      0     0   
28   DT50       0      0       0   0      0       0   0     0      0     0   
29   DT50       0      0       0   0      0       0   0     0      0     0   

    11      12  
0    0  238753  
1    0  716259  
2    0  716259  
3    0  716259  
4    0  716259  
5    0  716259  
6    0  716259  
7    0  716259  
8    0  716259  
9    0  716259  
10   0  238753  
11   0  716259  
12   0  716259  
13   0  716259  
14   0  716259  
15   0  716259  
16   0  716259  
17   0  716259  
18   0  716259  
19   0  716259  
20   0       0  
21   0       0  
22   0       0  
23   0       0  
24   0       0  
25   0       0  
26   0       0  
27   0       0  
28   0       0  
29   0       0